 Нам предстоит соскрейпить всю интересную информацию с [**knowyourmeme.com**](https://knowyourmeme.com).  
 Решение задачи, на данный момент, можно разбить на две части:
 #### 1.  Получение ссылок со страниц.  
 Сайт загружает по 16 мемов на страницу, необходимо вытащить на них ссылки, а также реализовать какой-нибудь механизм загрузки других страниц.
 
 #### 2.  Получение информации со страницы мема  
 Попробуем спарсить всю статистику по мему, а также ссылку на картинку и его текстовое описание.


## 1. Получение ссылок со страницы
 ### 1.1. Первый запрос
  Подключим библиотеку Requests. Она позволяет выполнять HTTP-запросы и обладает очень простым синтаксисом. 

In [1]:
import requests

Попробуем произвести наш первый запрос к серверу. В Requests GET-запрос выполняется следующим образом. Чтобы убедиться, что все прошло хорошо, выведем статус-код.

In [2]:
url = 'http://knowyourmeme.com/memes'
request = requests.get(url)
request.status_code

403

Однако, вместо желанного 200 (OK) мы получили 403 (Forbidden). Связано это с тем, что сервер посмотрел на наш User Agent и распознал в нас бота. Для того, чтобы обойти это, импортнем ```UserAgent``` из библиотеки ```fake_useragent```. Затем создадим словарь, в котором сгенерим *человеческого* юзер-агента.

In [3]:
from fake_useragent import UserAgent

In [4]:
ua = UserAgent()
header = {'User-Agent':str(ua.chrome)}
request = requests.get(url, headers=header)
request.status_code

200

Отлично, на этот раз все прошло успешно. Впредь будем использовать этот словарь каждый раз, когда будем делать запрос к серверу.

## 1.2. Парсинг HTML с помощью Beautiful Soup  
Успешно выполнив запрос с помощью ```requests``` мы получаем допуступ к html-коду страницы. Писать свой парсер мы, конечно, не будем, посколько все уже изобретено до нас. Например, библиотека Beautiful Soup. Она позволяет искать нужные тэги и извлекать из них информацию. То, что нам  нужно.  
Импортируем ```BeautifulSoup``` из ```bs4```.


In [5]:
from bs4 import BeautifulSoup

Для того, чтобы начать что-то искать на странице, нужно создать экземпляр класса BeautifulSoup. Итак, *сварим суп*, и посмотрим на его содержимое:

In [6]:
soup = BeautifulSoup(request.text)
soup

<!DOCTYPE html>
<html xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"c1a6d52f38","applicationID":"31165848","transactionName":"dFdfRUpeWglTQB8GDUNKWFRLHl8LUldI","queueTime":0,"applicationTime":59,"agent":""}</script>
<script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={licenseKey:"c1a6d52f38",applicationID:"31165848"};window.NREUM||(NREUM={}),__nr_require=function(n,e,t){function r(t){if(!e[t]){var i=e[t]={exports:{}};n[t][0].call(i.exports,function(e){var i=n[t][1][e];return r(i||e)},i,i.exports)}return e[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<t.length;i++)r(t[i]);return r}({1:[function(n,e,t){function r(){}function i(n,e,t){return function(){return o(n,[u.now()].concat(f(ar

Попробуем найти первый тэг ```<a>```.

In [7]:
soup.find('a')

<a href="/" title="Know Your Meme"></a>

Для того, чтобы получить содержимое тэга тоже ничего придумывать не надо. Достаточно обратиться к полю text. В этом случае поле пустое.

In [8]:
soup.find('a').text

''

Однако, что есть "полезного" в этом тэге - содержимое его аттрибута ```title```. Его можно получить, вызвав ```get(attr)```, где ```attr``` - название аттрибута.

In [9]:
soup.find('a').get('title')

'Know Your Meme'

Применим полученные знания для того, чтобы получить ссылки на мемы.  
Что нам нужно?
* Найти все теги ```<a>```, ведущие на страницы мемов
* Вытащить ссылку из аттрибута ```href```.  
Изучив код страницы с помощью браузера, выясним, что все тэги ```<a>```, которые ведут на желанные мемы, имеют аттрибут ```class = "photo"```.

Переберем все тэги ```<a>``` на странице и, если они имеют нужный аттрибут, сохраним ссылку в список.

Конечно, этот код можно было уместить в одну строку генератором, но оставим "развернутый" код.

In [10]:
url = 'http://knowyourmeme.com'
memes_links = []
for a in soup.find_all('a'):
    if a.get('class') == ['photo']:
        link = url + a.get('href')
        memes_links.append(link)
        
for link in memes_links:
    print(link)

http://knowyourmeme.com/memes/surprised-pikachu
http://knowyourmeme.com/memes/cursed-emojis
http://knowyourmeme.com/memes/folgers-brother-and-sister-commercial
http://knowyourmeme.com/memes/blini-cat
http://knowyourmeme.com/memes/subcultures/petscop
http://knowyourmeme.com/memes/short-tyler1
http://knowyourmeme.com/memes/epic-handshake
http://knowyourmeme.com/memes/ok-boomer
http://knowyourmeme.com/memes/double-ds-facts-book
http://knowyourmeme.com/memes/dont-dead-open-inside
http://knowyourmeme.com/memes/fat-yoshi
http://knowyourmeme.com/memes/marisaface-kanmarisa
http://knowyourmeme.com/memes/woman-yelling-at-a-cat
http://knowyourmeme.com/memes/subcultures/joker-2019-film
http://knowyourmeme.com/memes/wow-hot-a-what-baabhabhiat
http://knowyourmeme.com/memes/dean-mccoppin-mug


Отлично! Большая часть работы выполнена. Осталось понять, как скрипт будет перемещаться от страницы к странице.
## 1.3. Перемещение между страницами  
Нетрудно заметить, что при скроллинге каждые 16 мемов подгружается новая страница с мемами и меняется url. Получается что, чтобы получить мемы с опредленной страницы, нужно просто указать ее номер после ```http://knowyourmeme.com/memes/page/```. Проверим нашу *теорию*, попробовав загрузить мемы с первых трех страниц.

In [11]:
page_template = 'http://knowyourmeme.com/memes/page/{}'
for page in range(1,4):
    # Formatting page url
    page_url = page_template.format(page)
    # Creating request
    r = requests.get(page_url, headers=header)
    # Brewing soup
    soup = BeautifulSoup(r.text)
    # Using generator to fetch all the links 
    memes_links = [url + a.get('href') for a in soup.find_all('a') if a.get('class') == ['photo']]
    print('\nPage {}:'.format(page))
    print('Found {} memes'.format(len(memes_links)))


Page 1:
Found 16 memes

Page 2:
Found 16 memes

Page 3:
Found 16 memes


На каждой странице найдено по 16 мемов, как и ожидалось, а значит, все работает!

Напишем функцию, будет принимать номер страницы и словарь, содержащий User Agent, и будет возвращать список ссылок на данной странице.

In [12]:
def get_links(page, header):
    '''
    Returns list of links to the memes on the page
        page: int
            number of the page for parsing
        header: dict
            headers distionary that will be passed in requests.get()
        memes_links: [str]
            list of links
    '''
    link = 'http://knowyourmeme.com/memes/all/page/{}'.format(page)
    # Sending GET request
    try:
        response = requests.get(link, headers=header)
    except:
        return []    
    if not response.ok:
        print('Error code in get_links:', response.status_code)
        return []
    # Brewing soup
    soup = BeautifulSoup(response.text, features='lxml')
    # Lambda to filter all <a> tags that have 'class' attribute set to 'photo'
    a_photo_filter = lambda tag: tag.name == 'a' and tag.get('class') == ['photo']
    # Creating list of links to the memes
    memes_links = ['http://knowyourmeme.com' + a.get('href') for a in soup.find_all(a_photo_filter)]
    return memes_links

# 2. Получение информации со страницы  
Итак, мы добрались до самого сложного и интересного.

## 2.1. Получение статистики  
На каждой (ну или почти на каждой - все может быть...) странице с мемом есть блок, содержащий его статистику с сайта. В нее входит количество просмотров, комментариев, а также загруженных фото и видео.  

Вытащить эти данные можно из тега ```<dd>``` с аттрибутом ```class``` с одним из следующих значений ```views, videos, photos, comments```. Аттрибут ```title``` будет хранить необходимую информацию в строке. Однако, в ней будут присуствовать буквы, а также большие числа будут разделены запятой.  
Хорошо бы это почистить, верно? Преобразуем строку в нижний регистр, и удалим все запятые. Затем разделим строку на два слова: первое - значение, второе - измеряемая статистика. Запишем полученную пару значение-ключ в словарь.  

Применим все это на примере мема с кричащей девушкой и котом.

In [13]:
link = 'https://knowyourmeme.com/memes/woman-yelling-at-a-cat'
r = requests.get(link, headers=header)
soup = BeautifulSoup(r.text)
stats_dict = {}
# Stats we're searching for
stats = ['views', 'videos', 'photos', 'comments']
for stat in stats:
    dd = soup.find('dd', attrs={'class':stat})
    # Some stats maybe not available on some pages,
    # so we need to protect ourselves from getting an error
    if dd:
        val_key_str = dd.get('title').lower().replace(',', '')
        value, key = val_key_str.split(' ')
        stats_dict[key] = int(value)
print(stats_dict)

{'views': 3877325, 'videos': 21, 'images': 358, 'comments': 34}


Все работает! Обернем все это в функцию для дальнейшего использования. Будем передавать в функцию уже готовый суп.

In [14]:
def get_stats(soup, stat):
    '''
    Return requested stat of a meme
        soup: bs4 soup
            soup of a meme
        stats: string
            stat name string: views/videos/photos/comments
        value: int
            stat of a meme
    '''
    dd = soup.find('dd', attrs={'class':stat})
    value = dd.find_next('a').text.replace(',', '') if dd else 0
    return int(value)

## 2.2. Получение основной информации  
Здесь я хочу вытащить название мема, его статус, происхождение, дату появления, тэги, дату добавления на сайт и дату последнего изменения.  

Вытащить название максимально просто - это первый тэг ```<h1>``` на странице. Найдем его и получим название с помощью ```.text```.

In [15]:
m_name = soup.find('h1')
m_name = m_name.text.strip() if m_name else ''
print(m_name)

Woman Yelling at a Cat


Вся информация по статусу мема, его происхождению и тэгам вынесена в колонку справа от текста с его историей. Она представляет собой тэг ```<aside>``` с аттрибутом ```class="left"```. Beautiful Soup позволяет вызывать те же функции для поиска к тэгам, найденным в супе. Мы можем использовать это и начать искать внутри ```<aside>```.  

Первым тэгом ```<dl>``` внутри будет тэг, содержащий категорию страницы. Запишем его содержимое в отдельную переменную.  

In [16]:
properties = soup.find('aside', attrs={'class':'left'})
m_category = properties.find('dl')
m_category = m_category.find_next()
m_category = m_category.text.strip() if m_category else ''
print(m_category)

Meme


Статус мема можно найти в первом тэге ```<dd>```.  

In [17]:
m_status = properties.find('dd')
m_status = m_status.text.strip() if m_status else ''
print(m_status)

Confirmed


Тип мема получим, найдя первый тэг ```<a class="entry-type-link" ...><a/>```.  

In [18]:
m_type = properties.find('a', attrs={'class':'entry-type-link'})
m_type = m_type.text.strip() if m_type else ''
print(m_type)

Exploitable


Beautiful Soup также позволяет искать тэги по тексту, который они содержат. Найдем первый тэг, содержащий в тексте ```\nYear\n```, а затем после него найдем следующую ссылку. В тексте этой ссылки будет год появления мема.  

In [19]:
m_year = properties.find(text='\nYear\n')
m_year = m_year.find_next('a') if m_year else ''
m_year = int(m_year.text.strip()) if m_year else ''
print(m_year)

2019


Аналогично тому, как мы нашли статус мема, найдем ```<dd>``` с классом ```class="entry_origin_link"```. Следующей ссылкой будет ссылка с текстом, обозначающим то, откуда пошел мем.  

In [21]:
m_origin = properties.find('dd', attrs={'entry_origin_link'})
m_origin = m_origin.find_next('a') if m_origin else ''
m_origin = m_origin.text.strip() if m_origin else ''
print(m_origin)

Twitter


Найдем тэги, которые добавлены к мему, таким же образом, как искали год зарождения мема - по содержимому тэга.  

In [22]:
m_tags = properties.find(text='\nTags\n')
m_tags = m_tags.find_next() if m_tags else ''
m_tags = m_tags.text.strip() if m_tags else ''
print(m_tags)

cat, food, dinner, chair, the real housewives of beverly hills, taylor armstrong, kyle richards, deadbefordeath, perpetualwinter, 69-, apple-trump, malibu beach party from hell, missingegirl, lc28__, salad cat, table cat, lady screaming at cat, smudge, ku


Вытащим дату из тэга ```abbr``` с классом ```class="timeago"```. Однако, среди найденных тэгов нашлось и что-то лишнее.

In [23]:
times = soup.find_all('abbr', attrs={'class':'timeago'})
len(times)

4

Нужные нам тэги будут содержать строки ```"Added"``` или ```"Updated"```.

In [24]:
for t in times:
    # Split parent's tag text
    time = t.parent.text.split('\n')
    # If added or updated found in list, get
    if 'Added' in time:
        added = t.get('title')
    elif 'Updated' in time:
        updated = t.get('title')

print(added)
print(updated)

2019-06-20T11:14:05-04:00
2019-11-24T02:06:55-05:00


Напишем функцию, которая будет возвращать словарь из полученных значений.

In [25]:
def get_properties(soup):
    '''
    Return dictionary containing properties of a meme
        soup: bs4 soup
            soup of a meme
        properties_dict: {str:[str/int]}
            dict of all useful properties
    '''
    # Creating return dict
    properties_dict = {}

    # Name
    m_name = soup.find('h1')
    m_name = m_name.text.strip() if m_name else ''
    properties_dict['name'] = m_name

    # Finding <aside> tag contaning needed information
    properties = soup.find('aside', attrs={'class':'left'})
    if properties:
        # Category
        m_category = properties.find('dl')
        m_category = m_category.find_next()
        m_category = m_category.text.strip() if m_category else ''
        properties_dict['category'] = m_category.lower()

        m_status = properties.find_next('dd')
        m_status = m_status.text.strip() if m_status else ''
        properties_dict['status'] = m_status.lower()

        m_type = properties.find('a', attrs={'class':'entry-type-link'})
        m_type = m_type.text.strip() if m_type else ''
        properties_dict['type'] = m_type.lower()

        m_year = properties.find(text='\nYear\n')
        m_year = m_year.find_next() if m_year else ''
        if m_year:
            m_year = int(m_year.text.strip()) if m_year.text.strip().isdigit() else m_year.text.strip().lower()
        properties_dict['year'] = m_year

        m_origin = properties.find('dd', attrs={'entry_origin_link'})
        m_origin = m_origin.text.strip() if m_origin else ''
        properties_dict['origin'] = m_origin.lower()

        m_tags = properties.find(text='\nTags\n')
        m_tags = m_tags.find_next() if m_tags else ''
        m_tags = m_tags.text.strip() if m_tags else ''
        properties_dict['tags'] = m_tags.lower()

    # Fetching dates
    times = soup.find_all('abbr', attrs={'class':'timeago'})
    for t in times:
        # Split parent's tag text
        time = t.parent.text.split('\n')
        # If added or updated found in list, get
        if 'Added' in time:
            properties_dict['added'] = t.get('title')
        elif 'Updated' in time:
            properties_dict['updated'] = t.get('title')
    return properties_dict

## 2.3. Получение текстового описания  
Весь текст находится в тэге ```<section>``` с тэгом ```class="bodycopy"```. Посмотрим его содержимое.

In [26]:
bodycopy = soup.find('section', attrs={'class':'bodycopy'})
bodycopy

<section class="bodycopy">
<h2 id="about">About</h2>
<p><strong>Woman Yelling at a Cat</strong> refers to a <a class="internal-link" href="/memes/memes">meme</a> format featuring a screen cap of <em>The Real Housewives of Beverly Hills</em> cast members Taylor Armstrong and Kyle Richards <a class="internal-link" href="/memes/meme-chimera">followed by a picture</a> of a confused-looking <a class="auto-link" href="/memes/cultures/cats">cat</a> sitting behind a dinner plate. The format gained significant popularity across the web in mid-June 2019 and the cat was later identified as <a class="internal-link" href="/memes/smudge-the-cat/">Smudge the Cat</a>.</p>
<h2 id="origin">Origin</h2>
<div id="nativo-entry-1"></div> <h4 id="taylor-armstrong-crying-and-pointing">Taylor Armstrong Crying and Pointing</h4>
<p>On December 5th, 2011, episode 14 "Malibu Beach Party From Hell" of season two of <em>The Real Housewives of Beverly Hills</em> reality TV series premiered in the United States.<sup cl

Краткое описание мема из секции About находится в первом тэге ```<p>```.

In [27]:
about = bodycopy.find('p')
about.text.strip()

'Woman Yelling at a Cat refers to a meme format featuring a screen cap of The Real Housewives of Beverly Hills cast members Taylor Armstrong and Kyle Richards followed by a picture of a confused-looking cat sitting behind a dinner plate. The format gained significant popularity across the web in mid-June 2019 and the cat was later identified as Smudge the Cat.'

В секции Origin (где-то ее название - History) описано происхождение мема. Найдем первый тэг, в тексте которого есть эти слова. Следующий тэг ```<p>``` будет содержать текст.

In [28]:
next_h = bodycopy.find(text='Origin') or bodycopy.find(text='History')
next_h.find_next('p').text.strip()

'On December 5th, 2011, episode 14 "Malibu Beach Party From Hell" of season two of The Real Housewives of Beverly Hills reality TV series premiered in the United States.[1] In the episode, cast member Taylor Armstrong cries during an argument, with cast member Kyle Richards attempting to calm her down. (shown below).'

Однако, не весь текст находится в одном параграфе. Связано это тем, что практически на каждой странице вперемешку с картинками. Будем двигаться по тегам внутри цикла, пока не наткнемся на тэг ```<h2>```, что будет означать окончание этой секции.

In [29]:
history = ''
while next_h and next_h.name != 'h2':
    if next_h.name == 'p':
        history += next_h.text.strip()
    next_h = next_h.find_next()
history

'On December 5th, 2011, episode 14 "Malibu Beach Party From Hell" of season two of The Real Housewives of Beverly Hills reality TV series premiered in the United States.[1] In the episode, cast member Taylor Armstrong cries during an argument, with cast member Kyle Richards attempting to calm her down. (shown below).On December 6th, Daily Mail published an article describing the events of the episode, posting a still of Armstrong pointing and crying and Richards hugging her (shown below).[2]On June 19th, 2018, Tumblr user deadbefordeath posted a photograph of a white cat with a confused expression sitting in a chair in front of a plate with vegetables in it, titling the post "he no like vegetals." The post gained over 50,300 likes and reblogs in one year.[3]Following the popularity of the reaction image, on May 27th, 2019, smudge_lord Instagram account was created for the cat,[13] with the cat\'s name revealed to be Smudge.Until May 1st, 2019, both images saw moderate use as reactions 

Весь остальной текст вытащим похожим образом. Однако, тут могут встретиться и другие заголовки, поэтому в этот раз остановимся когда дойдем до заголовка с названием Search Interest.

In [30]:
next_o = next_h.find_next('p') if next_h else ''
other = ''
while next_o and (next_o.name != 'h2' and next_o.text != 'Search Interest'):
    if next_o.name == 'p':
        other += next_o.text.strip()
    next_o = next_o.find_next()
other

'On May 2nd, Twitter user @lc28__ made the first known meme based on the format, gaining over 20 retweets and 180 likes (shown below, left).[7] On June 2nd, 2019, Redditor PerpetualWinter made the first known object-labeling meme based on the format, gaining over 310 upvotes in one month (shown below, right).[8]On June 9th, Redditor 69- posted a template for the format to /r/memes subreddit.[9] On the same day, Redditor Apple-Trump posted a meme based on the format which gained over 38,600 upvotes in twelve days (shown below).[10]In the following days, the format gained significant spread on Reddit, with several highly-upvoted posts appearing in /r/dankmemes[11][12] and other subreddits. On November 4th, EbaumsWorld[14] published a collection of notable examples titled "49 Best \'Woman Yelling at a Cat\' Memes That Have Taken Over the Internet."On October 13th, 2019, Facebook user KucingMenangid posted a remix video of the Woman Yelling At Cat meme that places the cat Thurston Waffles 

В тексте часто встречаются ссылки в квадратных скобках, а также сноски в круглых. Поскольку нам этого не нужно, напишем функцию, которая будет удалять это удалять. Сделаем ее вложенной в фукнции, которая будет возвращать словарь с текстовой информацией. 

In [31]:
def get_text(soup):
    '''
    Returns dictionary of text info
        soup: bs4 soup
            soup of a meme
        text_dict: {str:str}
            dictionary of text info
    '''
    def remove_brackets(string, brackets):
        opening, closing = brackets
        # If deleting brackets
        if brackets == '[]':
            start = string.find(opening)
            end = string.find(closing)
            while start != -1 and end != -1:
                string = string.replace(string[start:end+1], '')
                start = string.find(opening)
                end = string.find(closing)
        # Else, deleting '(shown below)'
        else:
            substring = '(shown below)'
            index = string.find(substring)
            while index != -1:
                string = string.replace(substring, '')
                index = string.find(substring)
        return string

    text_dict = {}
    bodycopy = soup.find('section', attrs={'class':'bodycopy'})
    if bodycopy:
        # About text
        about = bodycopy.find('p')
        about = about.text.strip() if about else ''
        about = remove_brackets(about, '()')
        #about = remove_brackets(about, '[]')
        text_dict['about'] = about

        # Origin or history
        next_h = bodycopy.find(text='Origin') or bodycopy.find(text='History')
        next_h = next_h.find_next('p') if next_h else ''
        history = ''
        while next_h and next_h.name != 'h2':
            if next_h.name == 'p':
                history += next_h.text.strip()
            next_h = next_h.find_next()
        history = remove_brackets(history, '()')
        #history = remove_brackets(history, '[]')
        text_dict['history'] = history

        # Every other text
        next_o = next_h.find_next('p') if next_h else ''
        other = ''
        while next_o and (next_o.name != 'h2' and next_o.text != 'Search Interest'):
            if next_o.name == 'p':
                other += next_o.text.strip()
            next_o = next_o.find_next()
        other = remove_brackets(other, '()')
        #other = remove_brackets(other, '[]')
        text_dict['other'] = other

    return text_dict

## 2.4. Получение ссылки на картинку
Сразу качать мемы мы не будем, поэтому просто вытащим ссылку на картинку.  
Мы будем брать картинку, которая является превью мема. Она находится в тэге ```<a>``` с аттрибутом ```class="photo left wide"``` или ```class="photo left"``` для более старых мемов. Сначала попоробуем найти картинку с первым аттрибутом, и потом, если ничего не нашли, попробуем найти по второму аттрибуту.  
Однако, с тэгом ```class="photo left"``` есть еще и картинки ленты вверху сайта, в которой показаны последние добавленные мемы. Все они ведут на страницу мема и имеют адрес ```/memes/...``` в тэге ```href```, поэтому их будет легко отфильтровать.

In [32]:
a = soup.find('a', attrs={'class':'photo left wide'})
if not a:
    for a in soup.find_all('a', attrs={'class':'photo left'}):
        if a.get('href')[:6] != '/memes':
            break
link = a.get('href') if a else ''
print(link)

https://i.kym-cdn.com/entries/icons/original/000/030/157/womanyellingcat.jpg


Уже почти по традиции, напишем отдельную функцию.

In [33]:
def get_pic_link(soup):
    '''
        soup: bs4 soup
            soup of a meme page
        link: str
            link to a pic
    '''
    a = soup.find('a', attrs={'class':'photo left wide'})
    if not a:
        for a in soup.find_all('a', attrs={'class':'photo left'}):
            if a.get('href')[:6] != '/memes':
                break
    link = a.get('href') if a else ''
    return link

# 3. Объединение всего в одну функцию
Напишем функцию, которая будет вызывать написанные ранее функции. Было бы удобно скрыть отправку запроса и варку супа внутрь самой функции, чтобы аргументамии были бы ссылка на мем и хедер для ```requests```.  
Возможно, стоит объяснить, зачем подавать в функцию хедер. Дело в том, что если каждый раз генерировать нового юзер-агента, для сервера это тоже станет подозрительно и мы опять получим 403, будто мы и вовсе не подменяли User Agent. И правда, вряд ли реальный человек каждый запрос будет отправлять с нового браузера. Поэтому необходимо сгенерировать юзер-агент один раз, а затем использовать его для каждого запроса.

In [34]:
def get_data(link, header):
    '''
    Scraps the data from link
        link: str
            link to the meme
        data: {str:str/int}
            Dict contaning row of dataframe
    '''
    try:
        response = requests.get(link, headers=header)
    except:
        return {}
    if not response.ok:
        print("Error code in get_data:", response.status_code)
        return {}
    soup = BeautifulSoup(response.text, features='lxml')
    # Creating return dict
    data = {}
    for stat in ['views', 'videos', 'photos', 'comments']:
        data[stat] = get_stats(soup, stat)
    # Updating with dicts returned by previously written functions
    data.update(get_properties(soup))
    data.update(get_text(soup))
    # Adding pic link
    data['picture'] = get_pic_link(soup)

    return data

Подключим ```pandas``` и посмотрим на то, чего мы достигли.

In [35]:
import pandas as pd

In [45]:
data = get_data('http://knowyourmeme.com/memes/woman-yelling-at-a-cat', header)
data['name']

'Woman Yelling at a Cat'

In [57]:
df = pd.DataFrame(columns=['name', 'category', 'status','year', 'added', 'updated',
                       'views', 'videos', 'photos', 'comments', 'tags', 'type',
                       'about', 'history', 'other'])
df = df.append(data, ignore_index=True)
df

,name,category,status,year,added,updated,views,videos,photos,comments,tags,type,about,history,other,origin,picture
0,Woman Yelling at a Cat,meme,confirmed,2019,2019-06-20T11:14:05-04:00,2019-11-24T02:06:55-05:00,3877325,21,358,34,"cat, food, dinner, chair, the real housewives ...",exploitable,Woman Yelling at a Cat refers to a meme format...,"On December 5th, 2011, episode 14 ""Malibu Beac...","On May 2nd, Twitter user @lc28__ made the firs...",twitter,https://i.kym-cdn.com/entries/icons/original/0...


# 4. Обход блокировок с помощью Tor
Казалось бы, все готово - можно написать какой-то простенький цикл и запускать скрипт. Однако, рано или поздно, мы превысим количество запросов и сервер нас забанит.  
Попробуем обойти это, начав скрейпить через Tor. На каждую ошибку будем менять IP и генерировать новый User Agent.  
Для работы с Tor подключим библиотеку ```stem```, а вернее импортнем оттуда классы ```Controller``` и ```Signal```, а для того, чтобы использовать открытый Tor как прокси, подлючим модули ```socket``` и ```socks```. Модуль ```time``` нужен для того, чтобы вызывать ```sleep``` на время, необходимое тору, чтобы сменить ip. Теперь запустим Tor.

In [50]:
from stem.control import Controller
from stem import Signal
import socket
import socks
import time

In [81]:
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

Напишем функцию, которая будет возвращать наш ip-адрес.

In [52]:
def check_ip():
    ip_url = 'https://api.ipify.org'
    r = requests.get(ip_url)
    if r.ok:
        return r.text

Если вывести результат функции, получим ip, отличный от нашего настоящего, а значит мы успешно подключились к тору.  

In [82]:
print(check_ip())

185.107.47.171


Проверим, работает ли наша функция ```get_data```. Используем новый хедер со сгенерированным User Agent.

In [68]:
header = {'User-Agent':str(ua.chrome)}
url = 'https://knowyourmeme.com/memes/doge'
data = get_data(url, header)
df = df.append(data, ignore_index=True)
df

,name,category,status,year,added,updated,views,videos,photos,comments,tags,type,about,history,other,origin,picture
0,Woman Yelling at a Cat,meme,confirmed,2019,2019-06-20T11:14:05-04:00,2019-11-24T02:06:55-05:00,3877325,21,358,34,"cat, food, dinner, chair, the real housewives ...",exploitable,Woman Yelling at a Cat refers to a meme format...,"On December 5th, 2011, episode 14 ""Malibu Beac...","On May 2nd, Twitter user @lc28__ made the firs...",twitter,https://i.kym-cdn.com/entries/icons/original/0...
1,Doge,meme,confirmed,2013,2013-07-24T16:29:55-04:00,2019-12-06T08:30:40-05:00,13112807,62,1666,918,"animal, dog, shiba inu, shibe, such doge, supe...",animal,"Doge is a slang term for ""dog"" that is primari...","The use of the misspelled word ""doge"" to refer...","On October 28th, 2010, a photo of Kabosu was s...",tumblr,https://i.kym-cdn.com/entries/icons/original/0...


Отлично, все работает! Теперь все это можно собирать в скрипт. Нужно будет написать основной цикл со сменой ip каждую ошибку.